In [ ]:
import os
from google.colab import output
from google.colab import files

file_selector = files.upload()

if not file_selector:
  print("No file selected.")

uploaded_file = next(iter(file_selector.values()))

try:
  if not uploaded_file.name.endswith(".xlsx"):
    print("Please select an .xlsx file.")
except:
  print("Please check file type")

with open("/content/data.xlsx", "wb") as f:
  f.write(uploaded_file)

# Print a success message
print("File uploaded and saved as data.xlsx.")



Saving CIE L1_Aug2023 semester_OAT data.xlsx to CIE L1_Aug2023 semester_OAT data.xlsx
Please check file type
File uploaded and saved as data.xlsx.


Verify that the excel file has been added to the current directory.

> Need to verify later if the pickle files of both @Sairam & @Swastik are added to the current directory

In [ ]:
import pandas as pd


full_dataframe = pd.read_excel('/content/data.xlsx')

In [ ]:
# Sairam 7 questions

questions = [
    "Give one/top reason why you want to take up this course",
    "What (one/top thing) do you expect to get out of this course?",
    "What does entrepreneurship mean to you?",
    "Can you give one example of any prototype you have built?",
    "Optional question: Are you excited to work on any ideas/opportunities as part of this course. If so please elaborate."
  ]



AI Component Detection Below

In [ ]:
!pip install --upgrade tensorflow

  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached keras-3.2.1-py3-none-any.whl (1.1 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.1
    Uninstalling tensorflow-2.15.1:
      Successfully uninstalled tensorflow-2.15.1


In [ ]:
#run this cell once before predicting values (loading the model)
#Accuracy of the model = 82-86%
import tensorflow
import pickle
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text  # Registers the ops.
import numpy as np

class BertEncoder(tensorflow.keras.layers.Layer):
    def __init__(self, preprocess_path, model_path, **kwargs):
        super(BertEncoder, self).__init__(**kwargs)
        self.preprocessor = hub.KerasLayer(preprocess_path)
        self.encoder = hub.KerasLayer(model_path, trainable=True)

    def call(self, inputs):
        encoder_inputs = self.preprocessor(inputs)
        outputs = self.encoder(encoder_inputs)
        pooled_output = outputs['pooled_output']  # [batch_size, 512].
        sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].
        return pooled_output, sequence_output

with tensorflow.keras.utils.custom_object_scope({'BertEncoder': BertEncoder}):
    # Load the model architecture and weights
    model = tensorflow.keras.models.load_model('/content/Model/model-bert-use-this.hdf5')

# Load model weights and optimizer state from pickle file
with open('/content/Model/model_weights_and_optimizer.pickle', 'rb') as f:
    weights, optimizer_config = pickle.load(f)

# Set model weights
model.set_weights(weights)

# Reconstruct optimizer from config
optimizer = tensorflow.keras.optimizers.Adam.from_config(optimizer_config)

print("Model Loaded")

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
y_main=[]
for i in questions:
  temp1=[]
  y_pred=model.predict(full_dataframe[i].fillna("NA"))
  for j in y_pred:
    temp1.append(j[0])
  y_pred
  y_main.append(temp1)

NameError: name 'model' is not defined

In [ ]:
y_main = [list(i) for i in zip(*y_main)]#transposing the matrix
ai_df=pd.DataFrame(y_main)

In [ ]:
ai_df # Contains the value of how probable the sentence was AI written

""


Natural Language Processing Below

In [ ]:
# Your output df will be like this I presume, please use the same variable names.
import numpy as np
# Creating a random one to test it out
nlp_df = pd.DataFrame(np.random.rand(100, 1))

print(nlp_df)

import pandas as pd
from joblib import load

# Load the vectorizer
loaded_vectorizer = load('/content/vectorizer.joblib')

# Load the model
loaded_model = load('/content/random_forest_model.joblib')

# Read the data from data.xlsx
data_df = full_dataframe #pd.read_excel('/content/CIE L1_Aug2023 semester_OAT data.xlsx')

# Assuming the text responses are in columns 7 onwards
text_columns = data_df.columns[7:]

# Iterate over each row and make predictions
for index, row in data_df.iterrows():
    # Concatenate all text responses into a single string https://www.canva.com/design/DAGCG5bJlhs/ZaWlO8EVoVCsPdDpJqnM0A/edit?utm_content=DAGCG5bJlhs&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton
    answers_row = ' '.join(row[text_columns].astype(str))

    # Transform the text using the loaded vectorizer
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Make prediction for the row
    prediction = loaded_model.predict(answers_row_vec)

    # Print the predicted status
    predicted_status = 'confirmed' if prediction[0] == 1 else 'not confirmed'
    print(f"Row {index + 1} - Predicted status: {predicted_status}")

In [ ]:
import pandas as pd
from joblib import load

# Load the vectorizer and model
loaded_vectorizer = load('/content/vectorizer.joblib')
loaded_model = load('/content/random_forest_model.joblib')

# Assume data_df is already defined and includes the required data
text_columns = data_df.columns[7:]

# List to store each row's data
data = []

# Iterate over rows in the dataframe
for index, row in full_dataframe.iterrows():
    # Vectorize the given row
    answers_row = ' '.join(row[text_columns].astype(str))
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Predict the probability of the confirmed class
    probability = loaded_model.predict_proba(answers_row_vec)[:, 1]

    # Append dictionary to list
    data.append({'probability': float(probability[0])})

# Create DataFrame from list of dictionaries
results_df = pd.DataFrame(data)

# Optionally, to see the first few rows of your DataFrame:
print(results_df.head())


   probability
0     0.667667
1     0.338087
2     0.698056
3     0.497621
4     0.275978


In [ ]:
print(results_array)

[[0.00000000e+00 6.67666667e-01]
 [1.00000000e+00 3.38087302e-01]
 [2.00000000e+00 6.98055556e-01]
 [3.00000000e+00 4.97620851e-01]
 [4.00000000e+00 2.75977633e-01]
 [5.00000000e+00 4.61694444e-01]
 [6.00000000e+00 3.13472222e-01]
 [7.00000000e+00 6.30047619e-01]
 [8.00000000e+00 3.86698413e-01]
 [9.00000000e+00 4.30380952e-01]
 [1.00000000e+01 4.79769841e-01]
 [1.10000000e+01 6.80000000e-01]
 [1.20000000e+01 6.24333333e-01]
 [1.30000000e+01 4.47021284e-01]
 [1.40000000e+01 4.09662393e-01]
 [1.50000000e+01 5.91301587e-01]
 [1.60000000e+01 4.16320707e-01]
 [1.70000000e+01 4.70063492e-01]
 [1.80000000e+01 2.08222222e-01]
 [1.90000000e+01 4.71347403e-01]
 [2.00000000e+01 5.36611111e-01]
 [2.10000000e+01 6.30010101e-01]
 [2.20000000e+01 5.83777778e-01]
 [2.30000000e+01 5.98515152e-01]
 [2.40000000e+01 5.63931818e-01]
 [2.50000000e+01 4.09714286e-01]
 [2.60000000e+01 5.50833333e-01]
 [2.70000000e+01 2.19730159e-01]
 [2.80000000e+01 4.37134199e-01]
 [2.90000000e+01 3.18015873e-01]
 [3.000000

Outputting to the Excel File

In [ ]:
import pandas as pd
import numpy as np


# I am dividing it here to normalize the score

ai_df.loc[:, "AI Component"] = ai_df.sum(axis=1) / 5
ai_df = ai_df[["AI Component"]]
print(ai_df)

In [ ]:
# prompt: Append the dataframes to the end of the first sheet irrespective of the name of the sheet of the excel file such that the dataframes are appended after the last column that has data on the excel file, don't replace any data on the sheet

import pandas as pd

# Append the dataframes to the end of the first sheet irrespective of the name of the sheet of the excel file such that the dataframes are appended after the last column that has data on the excel file, don't replace any data on the sheet
def append_dataframes_to_excel(excel_file_path, dataframes):
  with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    for dataframe in dataframes:
      dataframe.to_excel(writer, sheet_name=writer.sheets.keys()[0], startrow=writer.sheets.values()[0].max_row, header=False, index=False)

# Example usage
excel_file_path = '/content/data.xlsx'
dataframes = [ai_df, nlp_df]
append_dataframes_to_excel(excel_file_path, dataframes)


Database

In [ ]:
import sqlite3 as s
import datetime

def write_2_db(c, connection, param, value):
  c.execute("""
    INSERT INTO GRAPH_DATA (PARAM, YEAR, MONTH, VALUE) VALUES (?, ?, ?, ?)
  """, (param, datetime.date.today().year, datetime.date.today().month, value))

  # Save the transaction
  connection.commit()

connection = s.connect("/content/base.sqlite")

c = connection.cursor()

# Create a new table
c.execute("""
    CREATE TABLE IF NOT EXISTS GRAPH_DATA (
        PARAM TEXT NOT NULL,
        YEAR INT NOT NULL,
        MONTH INT NOT NULL,
        VALUE TEXT NOT NULL
    )
""")

write_2_db(c, connection, "AI", 122)

# Read from database
c.execute("SELECT * from GRAPH_DATA")
print(c.fetchall())


In [ ]:
type(uploaded_file)